In [1]:
from tkinter import *
from tkinter.filedialog import askopenfilename 
from PIL import Image, ImageTk
import cv2
import os
import numpy as np


# Some Variables and the image loaded

In [25]:
ref_height = 92.5
#------ briey
start_x = 770
start_y = 300
stop_x = 770
stop_y = 690
#------ zoom images ------
#start_x = 621
#start_y = 497
#stop_x = 621
#stop_y = 947
#------ zoom0 images -----
#start_x = 758
#start_y = 437
#stop_x = 758
#stop_y = 705


imgSizeX =1700
imgSizeY= 1000


start_image =  cv2.imread('images/briey/28_5.jpeg') 



### Turn image as grayscal and risize it

In [26]:
#resize the images and turn to gray scale
gray_image = cv2.resize(cv2.cvtColor(start_image, cv2.COLOR_BGR2GRAY), (imgSizeX,imgSizeY), interpolation = cv2.INTER_AREA)
#gray_image = cv2.GaussianBlur(gray_image, (2,2), 1)
#cv2.imshow('grayscale', gray_image)
#cv2.imwrite('resized22cm.jpg', gray_image)

#cv2.waitKey(0)
#cv2.destroyAllWindows()


## Isolate the part of image we have interrest

In [ ]:
#DEBUG : draw crop rectangle
toDebug = cv2.rectangle(gray_image, (start_x,start_y), (stop_x+12,stop_y), (0,255,0),2)
cv2.imshow("debug", toDebug)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
lineWidth = 12
crop_image = gray_image[start_y:stop_y, start_x-round(lineWidth/2):start_x+round(lineWidth/2)]
cv2.imshow("cropped", crop_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Pass the interrested part of the image into filter

### contrast up

In [28]:

#cv2.imshow("no Thresh", crop_image)

#augmented contrast
#contrast_up_image = cv2.equalizeHist(crop_image)
contrast_up_image = crop_image

      

#cv2.imshow("more contrasted", contrast_up_image)

#cv2.waitKey(0)
#cv2.destroyAllWindows()

### filter

In [29]:

#thresh1 = cv2.adaptiveThreshold(contrast_up_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,51,21) #51,21
thresh1 = cv2.adaptiveThreshold(contrast_up_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,51,11) #51,21

cv2.imshow("cropped", crop_image)
cv2.imshow("thresholded",thresh1)

sobelx = cv2.Sobel(thresh1,cv2.CV_64F,0,1,ksize=5)


cv2.imshow("sobel",sobelx)
#_ ,sobelx_thresh = cv2.threshold(sobelx, 50, 255, cv2.THRESH_BINARY )
#cv2.imshow("sobel_thresh", sobelx_thresh)


#laplacian = cv2.Laplacian(thresh1,cv2.CV_64F)
#cv2.imshow("laplacian",laplacian)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Calculate the median value of snow height in pixel

In [30]:
sizeX = sobelx.shape[0]
sizeY = sobelx.shape[1]
#print(sizeX)
#print(sizeY)

sumPixel = []
for i in range(0,sizeY):
    countPixel = 0
    for j in range(1,sizeX):
        if sobelx[sizeX-j][i] == 0.0 :
            countPixel = countPixel + 1
        else:
            sumPixel.append(countPixel)
            break
snow_height_pixel = np.median(sumPixel)
print("Mediane de la hauteur [pixel] :"+str(snow_height_pixel))

Mediane de la hauteur [pixel] :118.0


In [31]:
reference_Height_mm = 1000


conversion_rapport = reference_Height_mm/sizeX
snow_height_mm = np.round(conversion_rapport * snow_height_pixel,1)
print("snow height pixel :" + str(snow_height_pixel))
print("rapport : " + str(np.round(conversion_rapport,1)) + "mm/pixel")
print("snow height mm " + str(snow_height_mm))

snow height pixel :118.0
rapport : 2.4mm/pixel
snow height mm 279.9


# Test result of algo